In [1]:
import numpy as np
import pandas as pd

In [69]:
#Load data
clothing_df = pd.read_csv('raw_data/clothes.csv')
clothing_df

,gender,product_category,product_name,details
0,women,JACKETS,oversize trench coat pocket,collared trench coat cotton blend feature long...
1,women,KNITWEAR,knit sweater zip,sweater viscose blend feature high neck metal ...
2,women,JEANS,marine straight fit highwaist pocket jean,slim fit relaxed leg high waistzw woman jeansh...
3,women,ACCESSORIES,crossbody bag,crossbody bag adjustable strap lining zip clos...
4,women,JEANS,marine straightleg highwaist jean,slim fit relaxed leg high waistzw woman jeansh...
...,...,...,...,...
3628,men,SHORTS,seersucker bermuda short,regularfit bermuda short elasticate waistband ...
3629,men,SHORTS,jogger waist bermuda short,regularfit bermuda short adjustable waistband ...
3630,men,SHORTS,bermuda short crochet pocket,bermuda short cotton viscose blend feature adj...
3631,men,SHORTS,linen viscose knit bermuda short,loosefitte knit bermuda short spun linen visco...


In [70]:
clothing_df['product_category'].unique()

array(['JACKETS', 'KNITWEAR', 'JEANS', 'ACCESSORIES',
       'T-SHIRTS_SWEATSHIRTS', 'TROUSERS', 'SHOES', 'SHIRTS',
       'WAISTCOATS_GILETS', 'SKIRTS', 'BLAZERS', 'T-SHIRTS',
       'SHORTS_SKORTS', 'DRESSES_JUMPSUITS', 'TOPS_BODYSUITS',
       'HOODIES_SWEATSHIRTS', 'SHORTS', 'POLO SHIRTS', 'OVERSHIRTS',
       'SWEATERS_CARDIGANS', 'SUITS', 'SWIMWEAR'], dtype=object)

In [71]:
#load the model
import torch
from transformers import pipeline

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cpu


In [72]:
temperature_mapping = {
            (5, 10): "5-10°C cold weather",
            (10, 15): "10-15°C cool weather", 
            (15, 20): "15-20°C mild weather",
            (20, 25): "20-25°C warm weather",
            (25, 30): "25-30°C hot weather"
        }

In [73]:
def temperature_to_label(temperature):
        """
        Convert numeric temperature to label
        """
        try:
            temp = float(temperature)
            for (low, high), label in temperature_mapping.items():
                if low <= temp <= high:
                    return label
            if temp < 5:
                return "5-10°C cold weather"
            elif temp > 30:
                return "25-30°C hot weather"
        except ValueError:
            raise ValueError(f"Invalid temperature: {temperature}. Please enter 5-30.")

In [81]:
temperature = '5'
temperature_label = temperature_to_label(temperature)
temp_range = temperature_label.split(' ')[0]

In [82]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

if len(clothing_df) > sample_size:
    sample_df = clothing_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(clothing_df)} total items")
else:
    sample_df = clothing_df
    print(f"🎯 Using all {len(sample_df)} items")

🎯 Sampling 5 random items from 3633 total items


In [83]:
# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)

In [84]:
# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = pipe(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This clothing item is suitable for {}.",
               multi_label=False
              )
results

⚡ Classifying sampled items in one batch...


[{'sequence': 'strapless dress.offtheshoulder short dress straightcut neckline strap detail metallic clip opening matching line invisible zip fastening',
  'labels': ['5-10°C cold weather'],
  'scores': [0.6541529893875122]},
 {'sequence': 'contrast trainer.trainers combination piece colour upper seveneyelet face contrastcoloure slightly chunky rubber sole',
  'labels': ['5-10°C cold weather'],
  'scores': [0.48742735385894775]},
 {'sequence': 'floral print swimming trunk.swimming trunk technical fabric adjustable elasticate waistband drawstre pocket patch pocket lininglength inner leg seam cm length outer leg seam cm view',
  'labels': ['5-10°C cold weather'],
  'scores': [0.5202308893203735]},
 {'sequence': 'contrast knit polo shirt.collar knit polo shirt contrast button fasten short sleeve rib trim',
  'labels': ['5-10°C cold weather'],
  'scores': [0.8925155997276306]},
 {'sequence': 'texture knit dress.short dress tie vneckline long sleeve',
  'labels': ['5-10°C cold weather'],
  

In [85]:
# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]

In [86]:
# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices

(array([0.65415299, 0.48742735, 0.52023089, 0.8925156 , 0.85108542]),
 array([3, 4, 0, 2, 1]))

In [87]:
# Build results
recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothing_df)} items"})
# print(recommendations[0]['product_name'])
# print(recommendations[1]['product_name'])
# print(recommendations[2]['product_name'])
recommendations

[{'rank': 1,
  'product_name': 'contrast knit polo shirt',
  'details': 'collar knit polo shirt contrast button fasten short sleeve rib trim',
  'confidence': np.float64(0.893),
  'temperature': '5°C',
  'temperature_range': '5-10°C',
  'sampled_from_total': '5/3633 items'},
 {'rank': 2,
  'product_name': 'texture knit dress',
  'details': 'short dress tie vneckline long sleeve',
  'confidence': np.float64(0.851),
  'temperature': '5°C',
  'temperature_range': '5-10°C',
  'sampled_from_total': '5/3633 items'},
 {'rank': 3,
  'product_name': 'strapless dress',
  'details': 'offtheshoulder short dress straightcut neckline strap detail metallic clip opening matching line invisible zip fastening',
  'confidence': np.float64(0.654),
  'temperature': '5°C',
  'temperature_range': '5-10°C',
  'sampled_from_total': '5/3633 items'},
 {'rank': 4,
  'product_name': 'floral print swimming trunk',
  'details': 'swimming trunk technical fabric adjustable elasticate waistband drawstre pocket patch po